## Preprocessing

In [3]:
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data (1).csv


In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ["EIN","NAME","STATUS","SPECIAL_CONSIDERATIONS"])

In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_type_count = application_df['APPLICATION_TYPE'].value_counts()
application_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_count[application_type_count<750].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts[classification_value_counts>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_value_counts[classification_value_counts<500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, 
                                columns = ['AFFILIATION','USE_CASE','ORGANIZATION','INCOME_AMT','APPLICATION_TYPE', 'CLASSIFICATION'], 
                                prefix = ['aff','use','org','inc','app','class'])
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,aff_CompanySponsored,aff_Family/Parent,aff_Independent,aff_National,aff_Other,aff_Regional,use_CommunityServ,use_Heathcare,...,app_T4,app_T5,app_T6,class_C1000,class_C1200,class_C2000,class_C2100,class_C3000,class_C7000,class_Other
0,5000,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,5000,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,6692,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,142590,1,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

<ipython-input-13-95d2e00f4d32>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(["IS_SUCCESSFUL"],1).values


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
print(X_train_scaled)

[[-0.02957055  1.08034442 -0.04278355 ... -0.24342285 -0.15414053
  -0.21275844]
 [-0.02957055 -0.92563074 -0.04278355 ... -0.24342285 -0.15414053
  -0.21275844]
 [-0.02957055 -0.92563074 -0.04278355 ... -0.24342285 -0.15414053
  -0.21275844]
 ...
 [-0.02957055  1.08034442 -0.04278355 ... -0.24342285 -0.15414053
  -0.21275844]
 [-0.02957055  1.08034442 -0.04278355 ... -0.24342285 -0.15414053
  -0.21275844]
 [-0.02957055  1.08034442 -0.04278355 ...  4.10807777 -0.15414053
  -0.21275844]]


## Compile, Train and Evaluate the Model

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=24, activation="relu", input_dim=38))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                936       
                                                                 
 dense_4 (Dense)             (None, 16)                400       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,353
Trainable params: 1,353
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 3s 2ms/step - loss: 0.5849 - accuracy: 0.7085
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7271
Epoch 3/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5551 - accuracy: 0.7310
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5537 - accuracy: 0.7289
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7295
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7309
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5513 - accuracy: 0.7312
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5506 - accuracy: 0.7319
Epoch 9/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7326
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accura

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5595 - accuracy: 0.7272 - 474ms/epoch - 2ms/step
Loss: 0.559458315372467, Accuracy: 0.7272303104400635


In [27]:
# Export our model to HDF5 file
nn.save('model_output.h5')

In [28]:
# Download File
files.download("model_output.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>